# 1. Installs, Imports and Settings

In [1]:
#!pip install spacy==3.1.1 #restart runtime after this
#!python -m spacy download en_core_web_sm

import csv
import pandas as pd
pd.options.display.max_rows
pd.set_option('display.max_colwidth', None)

import string
import operator
from itertools import islice
from collections import Counter

#!pip install nltk
from nltk import ngrams
import nltk as nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet');

import spacy
nlp = spacy.load('es_core_news_lg')

OSError: [E050] Can't find model 'es_core_news_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

#  2. Reading files 

In [2]:
df = pd.read_csv('data/alertas.csv', sep="|")
df = df[df["Text"].notnull()]
print("Size of Data:", len(df))
print()
print("Columns : ", list(df.columns))

Size of Data: 666

Columns :  ['Filename', 'Text', 'Type', 'Year']


# 3. Functions

In [3]:
import re
exceptions = [
    "SAT \n \nSede Central Calle 55 Nº 10 32 Of 115 \nTels 3147300 Ext 2437 Telefax 6915300 \nEmail", 
    "DE LA", 
    "NI OTRO", 
    "Tels 3147300 Ext 2437 Telefax 6915300", 
    "Ext 2437 Telefax 6915300", 
    "Tels 3147300 Ext 2437 Telefax 6915300", 
    "Sede Central Calle 55 No", 
    "Tels 3147300 Ext 2437 Telefax 6915300", 
    "SAT Sede Central Calle 55 Nº 10 32 Of 115 Email", 
    "SAT Sede Central Calle 55 Nº 1032 Of 115 Email", 
    "SA Email",
    "SAT INFORME DE RIESGO No",
    "DEL CONFLICTO ARMADO Sistema",
    "LA VIDA LA LIBERTAD Y",
    "DELEGADA PARA LA EVALUACIÓN DEL RIESGO POBLACIÓN CIVIL",
    "SAT Sede Central Calle 55 Nº 1032 Of 115 Tels 3147300 Ext 2437 Telefax 6915300 Email", 
    "SAT Sede Central Calle 55 Nº 10 32 Of 115 Tels 3147300 Ext 2437 Telefax 6915300 Email", 
    "INFORME DE RIESGO No", 
    "LA INTEGRIDAD FISICA DE LA POBLACIÓN", 
    "Tels 3147300 Ext 2437 Telefax", 
    "DELEGADA PARA LA EVALUACIÓN", 
    "LA POBLACIÓN CIVIL", 
    "Tels 3147300 Ext 2437 Telefax 6915300 Correo Electrónico", 
    "VILLEGAS Defensor Delegado",
    "Tels 3147300 Ext 2452 Telefax",
    "2437 Telefax",
    "CHACÓN Defensor Delegado", 
    "Tels 3147300 ext 24372464 Fax ext 2452 Bogotá",
    "Tels 3147300 Ext 2437 Telefax ext", 
    "Tels 3147300 Ext 2452 Telefax", 
    "Telefax Ext 2452 Correo Electrónico", 
    "2437 Telefax", 
    "Tels 3147300 ext 24372464 Fax ext 2452 Bogotá", 
    "2464 Fax ext 2452 Bogotá",
    "X", 
    "Inminencia alta Urgente Grado 2 Urgencia intermedia Grado 3 No", 
    "SAT Sede Central Calle 55 Nº 1032 Bloque C Tercer piso Tels 3147300 Ext 2437 Telefax", 
    "O SITUACIÓN CRÓNICA",
    "AUTORIDADES VINCULADAS AL", 
    
    
]
def cleantext1(text):
    text = text.lower()
    text = text.replace("\n", " ")
    for s in exceptions:
        text = text.replace(s.lower(), " ")
    text = re.sub(' +', ' ', text)
    return text    

df["Text"] = df["Text"].apply(cleantext1)

In [4]:
nlp.max_length = len
stop_words = stopwords.words('spanish') + ["at", "ala", "026", "1", "27", "06", "18", "www", "01", 
                                           "8000914814", "i", "t", "s", "wwwdefensoriagovco", "0", 
    "sede central", 
    "nº", 
    "calle 55", 
    "1032",
    "10", "32", "2", "email", "defensatyahoocom", "115", "3", "4", "", "5", "6", "7", "8", "9", "11", "19" ]

punct_signs = list(string.punctuation)
punct_signs += ['…','¿','•','”','“','–','&','∑','[',']', '▪', "I ’m", "️❤", "⠀ ⠀ ⠀", "⠀ ⠀", "¡", "·"]

sim_dict = {

}

emojies = ["🤚","🙋","😳","🏻‍", "♀", "🏻‍", "📈", "️ ⬆", "❤", "🏼", "💜", "👏"]

def replace_simliars(text, sim_dict):
    for key in sim_dict.keys():
        for v in sim_dict[key]:
            text = text.replace(v, key)
    return text

#clean stopwords and punctuation, lemmatization
def clean_text(text):   
    clean_text = []
    text = replace_simliars(text, sim_dict)
    for p in punct_signs:
        text = text.replace(p, ' ') 
        
    #for e in emojies:
     #   text = text.replace(e, ' ') 
    
    clean_text = text.lower().split()
    clean_text = [w for w in clean_text if w not in stop_words]
    
    nlp.max_length = len(' '.join(clean_text)) + 100
    doc = nlp(' '.join(clean_text))
    txt = []
    for token in doc:  
        if token.lemma_ not in stop_words and token.lemma:
            txt.append(token.lemma_)
    return txt

#m is the number of top ngrams.
def getNGrams(text, n, m):
    h_dict = {}   
    ngramas = list(ngrams(text, n))
    for grams in ngramas:
        words = ' '.join(grams).strip()
        if words not in h_dict:
            h_dict[words] = 0
        h_dict[words] = h_dict[words] + 1
        
    sorted_dict = sorted(h_dict.items(), key=operator.itemgetter(1), reverse=True)
    top_values = list(islice(sorted_dict, m))
    return [(r[0], r[1], round(r[1]/len(ngramas)*100,2)) for r in top_values]

def printNgrams(data, m):
    Ngrams = []
    unigrams = getNGrams(data, 1, m)
    bigrams = getNGrams(data, 2 , m)
    trigrams = getNGrams(data, 3 , m)
    for i in range(0,m):
        Ngrams.append(unigrams[i] + bigrams[i] + trigrams[i])
    df = pd.DataFrame(Ngrams, columns=['Unigrams', 'Absolute Freq', 'Relative Freq', 
                                       'Bigrams', 'Absolute Freq', 'Relative Freq', 
                                       'Trigrams', 'Absolute Freq', 'Relative Freq',]) 
    return df

# 4. Top 20 Ngrams 

In [5]:
years = list(df[df["Type"] == "advertencia"]['Year'].unique())
years = sorted(years)
years

[2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2018,
 2019,
 2020,
 2021,
 2022]

## 4.1 Top 20 Ngrams in Advertencias by Year

In [6]:
for y in years:
    dfg = df[(df["Type"] == "advertencia") & (df["Year"] == y)]
    print(y,"--", len(dfg), "documents")
    dt = printNgrams(clean_text(' '.join(dfg['Text'])), 20)
    display(dt)

2002 -- 27 documents


,Unigrams,Absolute Freq,Relative Freq,Bigrams,Absolute Freq,Relative Freq,Trigrams,Absolute Freq,Relative Freq
0,armado,165,0.93,actor armado,82,0.46,sede central calle,79,0.45
1,población,159,0.90,sede central,79,0.45,central calle 55,79,0.45
2,auc,149,0.84,central calle,79,0.45,temprano sat sede,73,0.41
3,riesgo,147,0.83,calle 55,79,0.45,sat sede central,73,0.41
4,municipio,147,0.83,alertar temprano,74,0.42,calle 55 of,67,0.38
5,zona,147,0.83,temprano sat,73,0.41,sistema alertar temprano,66,0.37
6,farc,140,0.79,sat sede,73,0.41,alertar temprano sat,64,0.36
7,fuerza,113,0.64,farc eln,69,0.39,farc eln auc,59,0.33
8,civil,106,0.60,55 of,67,0.38,actor armado ilegal,28,0.16
9,comando,105,0.59,sistema alertar,66,0.37,zona farc eln,27,0.15


2003 -- 58 documents


,Unigrams,Absolute Freq,Relative Freq,Bigrams,Absolute Freq,Relative Freq,Trigrams,Absolute Freq,Relative Freq
0,población,558,1.11,población civil,208,0.41,sede central calle,118,0.24
1,municipio,549,1.09,actor armado,182,0.36,central calle 55,118,0.24
2,armado,543,1.08,farc eln,138,0.27,bogotá dc colombia,116,0.23
3,riesgo,468,0.93,fuerza público,134,0.27,farc eln auc,113,0.23
4,zona,419,0.83,homicidio selectivo,133,0.27,sat sede central,109,0.22
5,auc,418,0.83,temprano sat,123,0.25,temprano sat sede,105,0.21
6,farc,397,0.79,calle 55,122,0.24,alertar temprano sat,91,0.18
7,civil,368,0.73,bogotá dc,120,0.24,sistema alertar temprano,89,0.18
8,comunidad,258,0.51,armado ilegal,119,0.24,actor armado ilegal,68,0.14
9,parte,256,0.51,sede central,118,0.24,armado ilegal zona,63,0.13


2004 -- 88 documents


...

,Unigrams,Absolute Freq,Relative Freq,Bigrams,Absolute Freq,Relative Freq,Trigrams,Absolute Freq,Relative Freq
0,municipio,3970,0.99,alertar temprano,1625,0.40,procesosubproceso prevención protección,1390,0.35
1,armado,3092,0.77,prevención protección,1602,0.40,prevención protección código,1390,0.35
2,grupo,2746,0.68,procesosubproceso prevención,1390,0.35,nombre procesosubproceso prevención,1389,0.35
3,población,2398,0.60,protección código,1390,0.35,alertar temprano versión,1315,0.33
4,temprano,2379,0.59,nombre procesosubproceso,1389,0.35,temprano versión vigente,1315,0.33
5,protección,2378,0.59,grupo armado,1381,0.34,versión vigente 07092018,1229,0.31
6,riesgo,2247,0.56,temprano versión,1315,0.33,vigente 07092018 página,1023,0.25
7,prevención,2057,0.51,versión vigente,1315,0.33,f10 alertar temprano,932,0.23
8,zona,2000,0.50,vigente 07092018,1229,0.31,protección código ppp01,765,0.19
9,nacional,1704,0.42,07092018 página,1109,0.28,código ppp01 f10,765,0.19


2021 -- 25 documents


,Unigrams,Absolute Freq,Relative Freq,Bigrams,Absolute Freq,Relative Freq,Trigrams,Absolute Freq,Relative Freq
0,municipio,2143,1.01,alertar temprano,828,0.39,procesosubproceso prevención protección,708,0.33
1,armado,1409,0.66,prevención protección,811,0.38,prevención protección código,708,0.33
2,grupo,1392,0.65,procesosubproceso prevención,708,0.33,temprano versión vigente,708,0.33
3,temprano,1324,0.62,protección código,708,0.33,versión vigente 07092018,708,0.33
4,protección,1268,0.60,temprano versión,708,0.33,nombre procesosubproceso prevención,707,0.33
5,riesgo,1247,0.59,versión vigente,708,0.33,f10 alertar temprano,649,0.31
6,población,1229,0.58,vigente 07092018,708,0.33,alertar temprano versión,649,0.31
7,prevención,1096,0.52,nombre procesosubproceso,707,0.33,vigente 07092018 página,597,0.28
8,zona,1082,0.51,f10 alertar,649,0.31,protección código ppp01,417,0.20
9,nacional,977,0.46,grupo armado,617,0.29,código ppp01 f10,417,0.20


2022 -- 14 documents


,Unigrams,Absolute Freq,Relative Freq,Bigrams,Absolute Freq,Relative Freq,Trigrams,Absolute Freq,Relative Freq
0,municipio,1133,0.78,prevención protección,584,0.40,nombre procesosubproceso prevención,510,0.35
1,grupo,1023,0.70,nombre procesosubproceso,510,0.35,procesosubproceso prevención protección,510,0.35
2,armado,967,0.66,procesosubproceso prevención,510,0.35,prevención protección código,276,0.19
3,riesgo,920,0.63,grupo armado,418,0.29,protección código ppp01,276,0.19
4,protección,879,0.60,alertar temprano,390,0.27,código ppp01 f10,276,0.19
5,prevención,764,0.52,armado ilegal,316,0.22,ppp01 f10 alertar,276,0.19
6,nacional,737,0.51,alerta temprano,282,0.19,f10 alertar temprano,276,0.19
7,población,731,0.50,protección código,276,0.19,alertar temprano versión,276,0.19
8,temprano,710,0.49,código ppp01,276,0.19,grupo armado ilegal,234,0.16
9,zona,624,0.43,ppp01 f10,276,0.19,temprano versión vigente,198,0.14


## 4.2 Top 20 Ngrams in Seguimientos by Year

In [7]:
for y in [2018, 2019]:
    dfg = df[(df["Type"] == "seguimiento") & (df["Year"] == y)]
    print(y,"--", len(dfg), "documents")
    dt = printNgrams(clean_text(' '.join(dfg['Text'])), 20)
    display(dt)

2018 -- 8 documents


,Unigrams,Absolute Freq,Relative Freq,Bigrams,Absolute Freq,Relative Freq,Trigrams,Absolute Freq,Relative Freq
0,nacional,278,0.99,plantilla vigente,88,0.31,grupo armado ilegal,38,0.14
1,riesgo,206,0.73,grupo armado,76,0.27,ntc gp1000 2009,32,0.11
2,acción,176,0.63,línea nacional,66,0.23,2008 ntc gp1000,28,0.10
3,municipio,168,0.60,derechos humanos,56,0.20,nacional plantilla vigente,28,0.10
4,medida,158,0.56,defensoría pueblo,48,0.17,línea nacional plantilla,26,0.09
5,armado,148,0.53,remitir información,48,0.17,plantilla vigente 27062017,22,0.08
6,pueblo,132,0.47,armado ilegal,44,0.16,alo derechos humanos,18,0.06
7,prevención,126,0.45,alerta temprano,38,0.14,laentidad remitir información,18,0.06
8,derechos,118,0.42,2008 ntc,38,0.14,medida mitigar riesgo,18,0.06
9,grupo,116,0.41,ntc gp1000,38,0.14,línea nacional 018000914814,16,0.06


2019 -- 1 documents


,Unigrams,Absolute Freq,Relative Freq,Bigrams,Absolute Freq,Relative Freq,Trigrams,Absolute Freq,Relative Freq
0,líder,287,2.09,derechos humanos,223,1.62,defensor derechos humanos,171,1.24
1,derechos,259,1.88,líder social,200,1.45,persona defensor derechos,107,0.78
2,social,240,1.74,defensor derechos,176,1.28,derechos humanos líder,98,0.71
3,humanos,230,1.67,persona defensor,132,0.96,humanos líder social,93,0.68
4,defensor,224,1.63,humanos líder,98,0.71,seguimiento persona defensor,76,0.55
5,persona,157,1.14,informe seguimiento,80,0.58,informe seguimiento persona,71,0.52
6,riesgo,119,0.86,seguimiento persona,76,0.55,líder social persona,26,0.19
7,informe,114,0.83,defensoría pueblo,61,0.44,social persona defensor,23,0.17
8,seguimiento,114,0.83,campesino agrario,46,0.33,campesino agrario pnis,23,0.17
9,caso,98,0.71,comunidad negro,36,0.26,comunitario campesino agrario,20,0.15


## 5. Exporting to html

In [1]:
!jupyter nbconvert --to html 5_NLP_Word_Tokenization.ipynb

[NbConvertApp] Converting notebook 5_NLP_Word_Tokenization.ipynb to html
[NbConvertApp] Writing 706150 bytes to 5_NLP_Word_Tokenization.html
